In [310]:
import os
import pandas as pd # type: ignore
import ast
import re

In [311]:
log_folder_path = '/Users/carlonue/Documents/Venvs/pathpave_pymoo/log_test'

In [312]:
def format_objective_values(log_file_path):
    
    # Read the log.csv of the current subfolder into a DataFrame
    df = pd.read_csv(log_file_path)
    df[" objectiveValues"] = df[" objectiveValues"].str.split("array")
    
    for x in range(len(df[" objectiveValues"])):
        del df[" objectiveValues"][x][0]
        for y in range(len(df[" objectiveValues"][x])):
            df[" objectiveValues"][x][y] = df[" objectiveValues"][x][y].replace("(", "").replace(")", "").replace("[", "").replace("]", "").replace(" ", "")
            df[" objectiveValues"][x][y] = df[" objectiveValues"][x][y].rstrip(',')
            parts = df[" objectiveValues"][x][y].split(',')
            df[" objectiveValues"][x][y] = [float(part.strip()) for part in parts]

    return df
        

In [313]:
def parse_tuples(s):
    tuples = re.findall(r'\((\d+), (\d+)\)', s)
    return [(int(x), int(y)) for x, y in tuples]

In [314]:
def format_paths(log_file_path):
    df = pd.read_csv(log_file_path)
    #print(len(df[" paths"]))
    #df[' paths'] = df[' paths'].apply(lambda x: ast.literal_eval(x))
    #print(df[" paths"])
    #print(type(df[" paths"][0][0]))

    df[' paths'] = df[' paths'].apply(parse_tuples)
    #print(df[' paths'])
    return df


In [317]:
# Going through all subfolders in the log folder calls the analyze_log_file function
for root, dirs, files in os.walk(log_folder_path):
    for dir_name in dirs:
        subfolder_path = os.path.join(root, dir_name)
        log_file_path = os.path.join(subfolder_path, 'log.csv')
        
        if os.path.isfile(log_file_path):
            
            # Call the analyze_log_file function Converts the messy objective values into list of list of floats
            df = format_objective_values(log_file_path)
            df = format_paths(log_file_path)
            #print(df[" paths"][0][0])
            


0      [(50, 25), (50, 24), (49, 24), (49, 25), (48, ...
1      [(50, 25), (50, 24), (49, 24), (48, 24), (47, ...
2      [(50, 25), (50, 26), (50, 25), (50, 24), (50, ...
3      [(50, 25), (49, 25), (50, 25), (50, 26), (50, ...
4      [(50, 25), (49, 25), (50, 25), (50, 26), (50, ...
                             ...                        
995    [(50, 25), (49, 25), (48, 25), (47, 25), (46, ...
996    [(50, 25), (49, 25), (48, 25), (47, 25), (46, ...
997    [(50, 25), (49, 25), (48, 25), (47, 25), (46, ...
998    [(50, 25), (49, 25), (48, 25), (47, 25), (46, ...
999    [(50, 25), (49, 25), (48, 25), (47, 25), (46, ...
Name:  paths, Length: 1000, dtype: object
0      [(50, 25), (50, 26), (49, 26), (49, 27), (49, ...
1      [(50, 25), (50, 26), (49, 26), (49, 27), (49, ...
2      [(50, 25), (50, 26), (49, 26), (49, 27), (49, ...
3      [(50, 25), (50, 24), (49, 24), (49, 23), (49, ...
4      [(50, 25), (50, 24), (49, 24), (49, 23), (49, ...
                             ...              

In [325]:
import os
import pandas as pd # type: ignore
from concurrent.futures import ThreadPoolExecutor, as_completed

def format_log_file(log_file_path):
    # Define the function to format the log file here
    df = format_objective_values(log_file_path)
    df = format_paths(log_file_path)
    return df

def process_subfolder(subfolder_path, output_folder):
    log_file_path = os.path.join(subfolder_path, 'log.csv')
    if os.path.isfile(log_file_path):
        df = format_log_file(log_file_path)
        
        # Save the DataFrame as a CSV file in the output folder
        subfolder_name = os.path.basename(subfolder_path)
        output_path = os.path.join(output_folder, f"{subfolder_name}_formatted.csv")
        df.to_csv(output_path, index=False)
        
        return output_path
    return None

def find_subfolders(log_folder_path):
    subfolders = []
    for root, dirs, files in os.walk(log_folder_path):
        for dir_name in dirs:
            subfolder_path = os.path.join(root, dir_name)
            subfolders.append(subfolder_path)
    return subfolders

def main(log_folder_path, output_folder):
    subfolders = find_subfolders(log_folder_path)

    # Use ThreadPoolExecutor for parallel processing
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = {executor.submit(process_subfolder, subfolder, output_folder): subfolder for subfolder in subfolders}
        
        results = []
        for future in as_completed(futures):
            try:
                result = future.result()
                if result:
                    results.append(result)
            except Exception as e:
                print(f"Error processing {futures[future]}: {e}")

    # Print the paths of the saved CSV files
    for result in results:
        print(f"Saved formatted CSV to: {result}")

if __name__ == "__main__":
    # Set your log folder path and output folder path here
    log_folder_path = "/Users/carlonue/Documents/Venvs/pathpave_pymoo/log_test"  # Replace with the actual path to your log folder
    output_folder = "/Users/carlonue/Documents/Venvs/pathpave_pymoo/csv_formatted"

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    main(log_folder_path, output_folder)


0      [(50, 25), (49, 25), (49, 24), (49, 23), (49, ...
1      [(50, 25), (49, 25), (49, 24), (50, 24), (50, ...
2      [(50, 25), (49, 25), (49, 24), (49, 23), (49, ...
3      [(50, 25), (49, 25), (49, 24), (49, 23), (49, ...
4      [(50, 25), (49, 25), (49, 24), (49, 23), (49, ...
                             ...                        
472    [(50, 25), (49, 25), (49, 24), (49, 23), (49, ...
473    [(50, 25), (49, 25), (49, 24), (49, 23), (49, ...
474    [(50, 25), (49, 25), (49, 24), (49, 23), (49, ...
475    [(50, 25), (49, 25), (49, 24), (49, 23), (49, ...
476    [(50, 25), (49, 25), (49, 24), (49, 23), (49, ...
Name:  paths, Length: 477, dtype: object
0       [(50, 25), (50, 24), (50, 25), (49, 25), (49, ...
1       [(50, 25), (50, 26), (49, 26), (49, 25), (50, ...
2       [(50, 25), (50, 26), (49, 26), (49, 25), (50, ...
3       [(50, 25), (50, 26), (49, 26), (49, 25), (50, ...
4       [(50, 25), (50, 26), (49, 26), (49, 25), (50, ...
                              ...         

In [328]:
df_new = pd.read_csv('/Users/carlonue/Documents/Venvs/pathpave_pymoo/csv_formatted/gradientObstacles_51_51_AGEMOEA_TwoPointCrossover_RadiusSamplingMutation_100_100000_RandomSampling_PathRepair_0_42_formatted.csv')
df_new.head()

,generation,map,width,height,algorithm,crossover,mutation,popsize,n_eval,samplingFunction,repairFunction,shiftingMethod,seed,objectiveValues,paths
0,0,gradientObstacles,51,51,AGEMOEA,TwoPointCrossover,RadiusSamplingMutation,100,100000,RandomSampling,PathRepair,random,42,"[array([921. , 640.66]), array([3067. , 2754....","[(50, 25), (50, 26), (49, 26), (49, 27), (49, ..."
1,1,gradientObstacles,51,51,AGEMOEA,TwoPointCrossover,RadiusSamplingMutation,100,100000,RandomSampling,PathRepair,random,42,"[array([921. , 640.66]), array([3067. , 2754....","[(50, 25), (50, 26), (49, 26), (49, 27), (49, ..."
2,2,gradientObstacles,51,51,AGEMOEA,TwoPointCrossover,RadiusSamplingMutation,100,100000,RandomSampling,PathRepair,random,42,"[array([921. , 640.66]), array([2561. , 767...","[(50, 25), (50, 26), (49, 26), (49, 27), (49, ..."
3,3,gradientObstacles,51,51,AGEMOEA,TwoPointCrossover,RadiusSamplingMutation,100,100000,RandomSampling,PathRepair,random,42,"[array([691. , 470.12]), array([717. , 648.0...","[(50, 25), (50, 24), (49, 24), (49, 23), (49, ..."
4,4,gradientObstacles,51,51,AGEMOEA,TwoPointCrossover,RadiusSamplingMutation,100,100000,RandomSampling,PathRepair,random,42,"[array([513. , 400.45]), array([139. , 94.7...","[(50, 25), (50, 24), (49, 24), (49, 23), (49, ..."


In [334]:
import os
import pandas as pd # type: ignore
import re
from concurrent.futures import ProcessPoolExecutor, as_completed

def format_objective_values(df):
    df[" objectiveValues"] = df[" objectiveValues"].str.split("array")
    
    for x in range(len(df[" objectiveValues"])):
        del df[" objectiveValues"][x][0]
        for y in range(len(df[" objectiveValues"][x])):
            df[" objectiveValues"][x][y] = df[" objectiveValues"][x][y].replace("(", "").replace(")", "").replace("[", "").replace("]", "").replace(" ", "")
            df[" objectiveValues"][x][y] = df[" objectiveValues"][x][y].rstrip(',')
            parts = df[" objectiveValues"][x][y].split(',')
            df[" objectiveValues"][x][y] = [float(part.strip()) for part in parts]

    return df

def parse_tuples(s):
    tuples = re.findall(r'\((\d+), (\d+)\)', s)
    return [(int(x), int(y)) for x, y in tuples]

def format_paths(df):
    df[' paths'] = df[' paths'].apply(parse_tuples)
    return df

def format_log_file(log_file_path):
    # Read the log.csv of the current subfolder into a DataFrame
    df = pd.read_csv(log_file_path)
    
    # Apply both formatting functions
    df = format_objective_values(df)
    df = format_paths(df)
    
    return df

def process_subfolder(subfolder_path, output_folder):
    log_file_path = os.path.join(subfolder_path, 'log.csv')
    if os.path.isfile(log_file_path):
        df = format_log_file(log_file_path)
        
        # Save the DataFrame as a CSV file in the output folder
        subfolder_name = os.path.basename(subfolder_path)
        output_path = os.path.join(output_folder, f"{subfolder_name}_formatted.csv")
        df.to_csv(output_path, index=False)
        
        return output_path
    return None

def find_subfolders(log_folder_path):
    subfolders = []
    for root, dirs, files in os.walk(log_folder_path):
        for dir_name in dirs:
            subfolder_path = os.path.join(root, dir_name)
            subfolders.append(subfolder_path)
    return subfolders

def main(log_folder_path, output_folder):
    subfolders = find_subfolders(log_folder_path)

    # Use ProcessPoolExecutor for parallel processing
    with ProcessPoolExecutor() as executor:
        futures = {executor.submit(process_subfolder, subfolder, output_folder): subfolder for subfolder in subfolders}
        
        results = []
        for future in as_completed(futures):
            try:
                result = future.result()
                if result:
                    results.append(result)
            except Exception as e:
                print(f"Error processing {futures[future]}: {e}")

    # Print the paths of the saved CSV files
    for result in results:
        print(f"Saved formatted CSV to: {result}")

if __name__ == "__main__":
    # Set your log folder path and output folder path here
    log_folder_path = "/Users/carlonue/Documents/Venvs/pathpave_pymoo/log_test"  # Replace with the actual path to your log folder
    output_folder = "/Users/carlonue/Documents/Venvs/pathpave_pymoo/csv_formatted"

    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    main(log_folder_path, output_folder)


Process SpawnProcess-53:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/concurrent/futures/process.py", line 251, in _process_worker
    call_item = call_queue.get(block=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'process_subfolder' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>


Error processing /Users/carlonue/Documents/Venvs/pathpave_pymoo/log_test/gradientObstacles_51_51_AGEMOEA_TwoPointCrossover_RadiusSamplingMutation_100_100000_RandomSampling_PathRepair_0_282126: A process in the process pool was terminated abruptly while the future was running or pending.
Error processing /Users/carlonue/Documents/Venvs/pathpave_pymoo/log_test/gradientObstacles_51_51_AGEMOEA_TwoPointCrossover_RadiusSamplingMutation_100_100000_RandomSampling_PathRepair_0_42: A process in the process pool was terminated abruptly while the future was running or pending.
Error processing /Users/carlonue/Documents/Venvs/pathpave_pymoo/log_test/meanderingRiverObstacles_51_51_DNSGA2_TwoPointCrossover_RadiusSamplingMutation_100_100000_RandomSampling_PathRepair_1_420: A process in the process pool was terminated abruptly while the future was running or pending.
Error processing /Users/carlonue/Documents/Venvs/pathpave_pymoo/log_test/meanderingRiverObstacles_51_51_CTAEA_TwoPointCrossover_RadiusSam

Process SpawnProcess-54:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/concurrent/futures/process.py", line 251, in _process_worker
    call_item = call_queue.get(block=True)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute 'process_subfolder' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>


In [342]:
df = pd.read_csv('/Users/carlonue/Documents/Venvs/pathpave_pymoo/csv_formatted/gradientObstacles_51_51_AGEMOEA_TwoPointCrossover_RadiusSamplingMutation_100_100000_RandomSampling_PathRepair_0_42_formatted.csv')
df.head()

,generation,map,width,height,algorithm,crossover,mutation,popsize,n_eval,samplingFunction,repairFunction,shiftingMethod,seed,objectiveValues,paths
0,0,gradientObstacles,51,51,AGEMOEA,TwoPointCrossover,RadiusSamplingMutation,100,100000,RandomSampling,PathRepair,random,42,"[[921.0, 640.66], [3067.0, 2754.5], [6827.0, 5...","[(50, 25), (50, 26), (49, 26), (49, 27), (49, ..."
1,1,gradientObstacles,51,51,AGEMOEA,TwoPointCrossover,RadiusSamplingMutation,100,100000,RandomSampling,PathRepair,random,42,"[[921.0, 640.66], [3067.0, 2754.5], [2561.0, 7...","[(50, 25), (50, 26), (49, 26), (49, 27), (49, ..."
2,2,gradientObstacles,51,51,AGEMOEA,TwoPointCrossover,RadiusSamplingMutation,100,100000,RandomSampling,PathRepair,random,42,"[[921.0, 640.66], [2561.0, 767.39], [691.0, 47...","[(50, 25), (50, 26), (49, 26), (49, 27), (49, ..."
3,3,gradientObstacles,51,51,AGEMOEA,TwoPointCrossover,RadiusSamplingMutation,100,100000,RandomSampling,PathRepair,random,42,"[[691.0, 470.12], [717.0, 648.03], [613.0, 408...","[(50, 25), (50, 24), (49, 24), (49, 23), (49, ..."
4,4,gradientObstacles,51,51,AGEMOEA,TwoPointCrossover,RadiusSamplingMutation,100,100000,RandomSampling,PathRepair,random,42,"[[513.0, 400.45], [139.0, 94.77], [507.0, 316....","[(50, 25), (50, 24), (49, 24), (49, 23), (49, ..."


In [343]:
type(df[" objectiveValues"][0])

str

In [344]:
df[" objectiveValues"] = df[" objectiveValues"].apply(ast.literal_eval)


In [346]:
type(df[" objectiveValues"][0][0][0])

float